In [1]:
import datetime
import time
from airflow import DAG
from airflow.models import Variable
from airflow.operators.python import PythonOperator , BranchPythonOperator
from airflow.operators.dummy import DummyOperator
import pandas

import psycopg2

In [ ]:
start_date = Variable.get('DAG_STARTDATE')
start_date = datetime.datetime.strptime(start_date + ' 09:00:00','%Y%m%d %H:%M:%S')

# Setting

## Arguments

In [1]:
args = {
    'owner' : 'Buneo' ,
    'depends_on_past' : False ,
    'start_date' : start_date ,
    'email_on_failure' : 'buneostock@gmail.com' ,
    'email_on_retry' : 'buneostock@gmail.com' ,
    'retreis' : 2 ,
    'retry_delay' : datetime.timedelta(minutes = 5)
        }

dag = DAG('stock_today_data',
          description = '' ,
          schedule_interval = '0 9 * * 1-5',
          default_args = args ,
          tags = ['stock'])

NameError: name 'start_date' is not defined

## Variables

In [3]:
postgres_password = Variable.get("Postgres_password")

## DataBase Connection

In [4]:
conn = psycopg2.connect(host = '127.0.0.1' , dbname = 'postgres' , user = 'postgres' , password = postgres_password)
cur = conn.cursor()

# WORK_DATE 

## 【 Function 】

In [5]:
def WORK_DATE():
# Truncate table work_date and reinsert date data
    work_D = []
    if datetime.date.today().weekday() in range(0,5):
        work_D.append(datetime.date.today())

    for D in work_D:

        insert_D = '{:0>4}-{:0>2}-{:0>2}'.format(D.year,D.month,D.day)   
        query_D = '{:0>3}/{:0>2}/{:0>2}'.format(D.year-1911,D.month,D.day)

        try:
            if pandas.read_html("https://www.tpex.org.tw/web/stock/aftertrading/index_summary/summary_print.php?l=zh-tw&d={}&s=0,asc,0".format(query_D))[0].iloc[2,0] != '':
                if pandas.read_sql("select * from work_date where date = '{}'".format(insert_D), con = conn).shape[0] == 0:
                    cur.execute("insert into work_date (date) values ('{}')".format(insert_D))
                    cur.execute('commit')
                    print(f'{insert_D} insert to work_date.')
                    time.sleep(3)
                else:
                    print('{} is exist.'.format(insert_D))
            else :
                print(f'{insert_D} failed.')
        except IndexError:
            print('{} No data'.format(insert_D))
            pass

## 【 PythonOperator 】

In [6]:
task_WORK_DATE = PythonOperator(
            task_id = 'task_WORK_DATE' ,
            python_callable = WORK_DATE , 
            dag = dag)

#  CHECK_WORK_DATE 

## 【 Function 】

In [7]:
def CHECK_WORK_DATE():
    lastest_date = pandas.read_sql("select date from work_date order by date desc", con = conn)['date'].iloc[0]
    
    if datetime.date.today() == lastest_date :
        return 'task_GET_STOCK_DAILY'
    else :
        return 'task_NOT_TRADED_DATE'

## 【 BranchPythonOperator 】

In [8]:
task_CHECK_WORK_DATE = BranchPythonOperator(task_id = 'task_CHECK_WORK_DATE',
                                            python_callable = CHECK_WORK_DATE,
                                            dag = dag)

# NOT_TRADED_DATE

## 【 DummpyOperator 】

In [9]:
task_NOT_TRADED_DATE = DummyOperator(task_id = 'task_NOT_TRADED_DATE' , 
                                     dag = dag)

# GET_STOCK_DAILY

## 【 Function 】

In [10]:
def GET_STOCK_DAILY():
    list_url = 'https://www.twse.com.tw/exchangeReport/MI_INDEX?response=html&date={:0>4}{:0>2}{:0>2}&type=ALLBUT0999'
    list_sql = 'insert into stock_daily (no,name,close,fluctuation,open,highest,lowest,quantity,date) \
                values (%s,%s,%s,%s,%s,%s,%s,%s,%s)'

    work_D = [pandas.read_sql("select date from work_date order by date desc ",con = conn)['date'][0]]
    
 
    for D in work_D:
     
        process_day = '{:0>4}-{:0>2}-{:0>2}'.format(D.year , D.month , D.day) 
        print(process_day)
        
        cur.execute("delete from stock_daily where date = '{}'".format(process_day))
        cur.execute("commit")
        
        try:
            stock = pandas.read_html(list_url.format(D.year , D.month , D.day))[8]
            stock = stock.iloc[:,[0,1,8,9,10,5,6,7,2]]
            stock.columns = ['no','name','close','fluc(+/-)','fluc','open','highest','lowest','quantity']
            stock['store'] = stock.apply(lambda x : 'y' if len(x['no']) == 4 else 'n' , axis = 1)
            stock = stock[stock['store'] == 'y']
            stock['date'] = process_day
            stock['quantity'] = round(stock['quantity']/1000,0)
            stock['close'] = stock['close'].str.replace('--','0')
            stock['open'] = stock['open'].str.replace('--','0')
            stock['highest'] = stock['highest'].str.replace('--','0')
            stock['lowest'] = stock['lowest'].str.replace('--','0')
            stock['fluctuation'] = stock.apply(lambda x : x['fluc']*(-1) if x['fluc(+/-)'] == '-' else x['fluc'] , axis = 1)
            stock = stock.iloc[:,[0,1,2,4,5,6,7,8,10]]
            
            tmp = pandas.read_sql("select * from stock_daily where date = '{}'".format(process_day),con = conn)
            
            i_table = pandas.merge(left = stock , right = tmp , how = 'left' , on = 'no')
            i_table = i_table[i_table['name_y'].isnull()].iloc[:,:9]
            i_table.columns = ['no','name','close','fluctuation','open','highest','lowest','quantity','date']
            
            for _,data in i_table.iterrows():
                cur.execute(list_sql,data)
            cur.execute('commit')           
            
            print('【 Stock Daily Data 】{} data inserted .'.format(process_day))
            stock = ''
            time.sleep(20)
            
        except ValueError:
            print('【 Stock Daily Data 】{} No data'.format(process_day))
            pass

## 【 PythonOperator 】

In [11]:
task_GET_STOCK_DAILY = PythonOperator(task_id = 'task_GET_STOCK_DAILY',
                                      python_callable = GET_STOCK_DAILY,
                                      dag = dag)

# GET_OTC_DAILY

## 【 Function 】

In [12]:
def GET_OTC_DAILY():
    
    otc_url = 'https://www.tpex.org.tw/web/stock/aftertrading/daily_close_quotes/stk_quote_result.php?l=zh-tw&o=htm&d={:0>3}/{:0>2}/{:0>2}&s=0,asc,0'
    otc_sql = 'insert into otc_daily (no,name,close,fluctuation,open,highest,lowest,quantity,date) \
               values (%s,%s,%s,%s,%s,%s,%s,%s,%s)'
    
    work_D = [pandas.read_sql("select date from work_date order by date desc ",con = conn)['date'][0]]    
    
    for D in work_D:
        
        process_day = '{:0>4}-{:0>2}-{:0>2}'.format(D.year , D.month , D.day)            
        process_otc = pandas.read_html(otc_url.format(D.year-1911 , D.month , D.day))[0]
        
        if process_otc.iloc[-1,0] != '共0筆':
            
            process_otc = process_otc.iloc[:-4,[0,1,2,3,4,5,6,8]]
            process_otc.columns = ['no','name','close','fluctuation','open','highest','lowest','quantity']
            process_otc = process_otc[process_otc['no'] != '管理股票']
            process_otc['quantity'] = round(process_otc['quantity'].astype(int)/1000,0).astype(int).astype(int)
            process_otc['close'] = process_otc['close'].str.replace('---','0')
            process_otc['open'] = process_otc['open'].str.replace('---','0')
            process_otc['highest'] = process_otc['highest'].str.replace('---','0')
            process_otc['lowest'] = process_otc['lowest'].str.replace('---','0')
            process_otc['date'] = process_day
            process_otc['store'] = ''
            process_otc['store'] = process_otc.apply(lambda x : 'Y' if len(x['no']) == 4 else 'N' , axis =1)
            process_otc = process_otc[process_otc['store'] == 'Y'].iloc[:,:-1]
            
            tmp = pandas.read_sql("select * from otc_daily where date = '{}'".format(process_day) , con = conn)
            i_table = pandas.merge(left = process_otc , right = tmp , how = 'left' , on = ['no'])            
            i_table = i_table[i_table['name_y'].isnull()].iloc[:,:9]
            
            for _,row_data in i_table.iterrows():
                cur.execute(otc_sql,row_data)
            cur.execute('commit')

            print('【 OTC Daily Data 】{} data inserted .'.format(process_day))
            process_otc =''
            time.sleep(10)
        else:
            print('【 OTC Daily Data 】{} No data '.format(process_day))
        


## 【 PythonOperator 】

In [13]:
task_GET_OTC_DAILY = PythonOperator(task_id = 'task_GET_OTC_DAILY',
                                    python_callable = GET_OTC_DAILY,
                                    dag = dag)

# GET_STOCK_DAILY_AVG

## 【 Function 】

In [14]:
def GET_STOCK_DAILY_AVG():

    list_url = 'https://www.twse.com.tw/exchangeReport/MI_INDEX?response=html&date={:0>4}{:0>2}{:0>2}&type=ALLBUT0999'
    avg_p_sql = 'insert into stock_daily_avg_price (date , no , price) values (%s,%s,%s)'
    
    work_D = [pandas.read_sql("select date from work_date order by date desc ",con = conn)['date'][0]]
    
    for D in work_D:
        
        process_day = '{:0>4}-{:0>2}-{:0>2}'.format(D.year , D.month , D.day)     
        
        try:        
            avg_stock = pandas.read_html(list_url.format(D.year,D.month,D.day))[8]
            avg_stock = avg_stock.iloc[:,[0,2,4]]
            avg_stock.columns = ['no','total_q','total_a']
            avg_stock['store'] = avg_stock.apply(lambda x : 'y' if len(x['no']) == 4 else 'n' , axis = 1)
            avg_stock = avg_stock[avg_stock['store']=='y']
            avg_stock['avg_price'] = round(avg_stock['total_a']/avg_stock['total_q'],2)
            avg_stock['date'] = process_day
            avg_stock = avg_stock[['date','no','avg_price']]
            
            tmp = pandas.read_sql("select * from stock_daily_avg_price where date = '{}'".format(process_day) , con = conn)
            
            # 判斷
            i_table = pandas.merge(left = avg_stock , right = tmp , how = 'left' , on = ['no'])           
            i_table = i_table[i_table['date_y'].isnull()].iloc[:,:3]
            
            for _,data in i_table.iterrows():
                cur.execute(avg_p_sql,data)
            cur.execute('commit')

            print('【 Stock Average price 】{} data inserted .'.format(process_day))
            time.sleep(20)
            
        except ValueError :
            print('【 Stock Average price 】{} No data'.format(process_day))
            pass
    

## 【 PythonOperator 】

In [15]:
task_GET_STOCK_DAILY_AVG = PythonOperator(task_id = 'task_GET_STOCK_DAILY_AVG' ,
                                          python_callable = GET_STOCK_DAILY_AVG ,
                                          dag = dag)

# GET_OTC_DAILY_AVG

## 【 Function 】

In [16]:
def GET_OTC_DAILY_AVG():
    
    otc_url = 'https://www.tpex.org.tw/web/stock/aftertrading/daily_close_quotes/stk_quote_result.php?l=zh-tw&o=htm&d={:0>3}/{:0>2}/{:0>2}&s=0,asc,0'
    avg_p_sql = 'insert into stock_daily_avg_price (date , no , price) values (%s,%s,%s)'
    
    work_D = [pandas.read_sql("select date from work_date order by date desc ",con = conn)['date'][0]]    
    
    for D in work_D:        
                
        process_day = '{:0>4}-{:0>2}-{:0>2}'.format(D.year , D.month , D.day)        
        
        process_otc = pandas.read_html(otc_url.format(D.year-1911 , D.month , D.day))[0]
        process_otc = process_otc.iloc[:-4,[0,7]]
        process_otc.columns = ['no','price']
        process_otc['date'] = process_day
        process_otc['store']=''
        process_otc['store'] = process_otc.apply(lambda x : 'n' if len(x['no']) != 4 or x['no'] == '管理股票' else 'y' , axis = 1)
        process_otc = process_otc[process_otc['store'] == 'y']
        process_otc= process_otc.iloc[:,[2,0,1]]
        
        tmp = pandas.read_sql("select * from stock_daily_avg_price where date = '{}'".format(process_day) , con = conn)
        
        i_table = pandas.merge(left = process_otc , right = tmp , how = 'left' , on = 'no')
        i_table = i_table[i_table['date_y'].isnull()].iloc[:,:3]
        
        for _,data in i_table.iterrows():
            cur.execute(avg_p_sql , data)
        cur.execute('commit')        

        print('【 OTC Average price 】{} data inserted .'.format(process_day))
        time.sleep(10)
 

## 【 PythonOperator 】

In [17]:
task_GET_OTC_DAILY_AVG = PythonOperator(task_id = 'task_GET_OTC_DAILY_AVG' ,
                                        python_callable = GET_OTC_DAILY_AVG ,
                                        dag = dag)

# GET_STOCK_FI_OBS

## 【 Function 】

In [18]:
def STOCK_FI_OBS():

    foreign_investor_stcok_url = 'https://www.twse.com.tw/fund/TWT38U?response=html&date={:0>4}{:0>2}{:0>2}'   
    foreign_investor_sql = 'insert into foreign_investor (date , no , quantity) values (%s,%s,%s)'
    
    work_D = [pandas.read_sql("select date from work_date order by date desc ",con = conn)['date'][0]]
    
    for D in work_D:
        
        process_day = '{:0>4}-{:0>2}-{:0>2}'.format(D.year , D.month , D.day)
    
        try:
            fi = pandas.read_html(foreign_investor_stcok_url.format(D.year,D.month,D.day))[0]
            fi = fi.iloc[:,[1,11]]
            fi.columns = ['no','quantity']
            fi['no'] = fi['no'].astype('string').str.pad(width = 4 , side = 'left' , fillchar = '0')
            fi['store'] = fi.apply(lambda x : 'y' if len(x.loc['no']) == 4 else 'n' ,axis =1)
            fi = fi[fi['store']=='y']
            fi['date'] = process_day
            fi = fi.iloc[:,[3,0,1]]
            
            tmp = pandas.read_sql("select * from foreign_investor where date = '{}'".format(process_day), con = conn)
            
            i_table = pandas.merge(left = fi , right = tmp , how = 'left', on = 'no')
            i_table = i_table[i_table['date_y'].isnull()].iloc[:,:3]

            
            for _ , data in i_table.iterrows():               
                cur.execute(foreign_investor_sql,data)
                cur.execute('commit')
            cur.execute('commit')
            
            
            print('【 Foreign Investor (Stock) Over Bought / Sold 】{} data inserted .'.format(process_day))
            time.sleep(20)
            
        except ValueError:
            print(' Foreign Investor (Stock) Over Bought / Sold 】{} No data'.format(process_day))
            pass

## 【 PythonOperator 】

In [19]:
task_GET_STOCK_FI_OBS = PythonOperator(task_id = 'task_GET_STOCK_FI_OBS',
                                       python_callable = STOCK_FI_OBS ,
                                       dag = dag)

# GET_OTC_FI_OBS

## 【 Function 】

In [20]:
def OTC_FI_OBS():
    
    foreign_investor_otc_buy_url = 'https://www.tpex.org.tw/web/stock/3insti/qfii_trading/forgtr_result.php?l=zh-tw&t=D&type=buy&d={:}/{:0>2}/{:0>2}&s=0,asc&o=htm'
    foreign_investor_otc_sell_url = 'https://www.tpex.org.tw/web/stock/3insti/qfii_trading/forgtr_result.php?l=zh-tw&t=D&type=sell&d={:}/{:0>2}/{:0>2}&s=0,asc&o=htm'
    foreign_investor_sql = 'insert into foreign_investor (date , no , quantity) values (%s,%s,%s)'
    
    work_D = [pandas.read_sql("select date from work_date order by date desc ",con = conn)['date'][0]]
    
    for D in work_D:
        
        process_day = '{:0>4}-{:0>2}-{:0>2}'.format(D.year , D.month , D.day)   

# Foreign investor (OTC) Over Bought

        fi_otc = pandas.read_html(foreign_investor_otc_buy_url.format(D.year-1911,D.month,D.day))[0]
        if fi_otc.iloc[-1,0] != '共0筆':
            fi_otc = fi_otc.iloc[:,[1,11]]
            fi_otc.columns = ['no','quantity']
            fi_otc['no'] = fi_otc['no'].astype('string').str.pad(width = 4 , side = 'left' , fillchar = '0')
            fi_otc['store'] = fi_otc.apply(lambda x : 'y' if len(x['no']) == 4 else 'n' ,axis =1 )
            fi_otc = fi_otc[fi_otc['store']=='y']
            fi_otc['quantity'] = fi_otc['quantity'].astype('int')*1000
            fi_otc['date'] = process_day
            fi_otc = fi_otc.iloc[:,[3,0,1]]
            
            tmp = pandas.read_sql("select * from foreign_investor where date = '{}'".format(process_day) , con = conn)

            i_table = pandas.merge(left = fi_otc , right = tmp , on = 'no' , how = 'left')
            i_table = i_table[i_table['date_y'].isnull()].iloc[:,:3]          
            
            for _ , data in i_table.iterrows():
                cur.execute(foreign_investor_sql,data)
            cur.execute('commit')
            
            i_table = ''
            
            print('【 Foreign Investor (otc) Over Bought 】{} data inserted .'.format(process_day))
        else :
            print('【 Foreign Investor (otc) Over Bought 】{} No data .'.format(process_day))
    
# Foreign investor (OTC) Over Sold       

        fi_otc = pandas.read_html(foreign_investor_otc_sell_url.format(D.year-1911,D.month,D.day))[0]
        if fi_otc.iloc[-1,0] != '共0筆':
            fi_otc = fi_otc.iloc[:,[1,11]]
            fi_otc.columns = ['no','quantity']
            fi_otc['no'] = fi_otc['no'].astype('string').str.pad(width = 4 , side = 'left' , fillchar = '0')
            fi_otc['store'] = fi_otc.apply(lambda x : 'y' if len(x['no']) == 4 else 'n' ,axis =1 )
            fi_otc = fi_otc[fi_otc['store']=='y']
            fi_otc['quantity'] = fi_otc['quantity'].astype('int')*1000
            fi_otc['date'] = process_day
            fi_otc = fi_otc.iloc[:,[3,0,1]]
            
            tmp = pandas.read_sql("select * from foreign_investor where date = '{}'".format(process_day) , con = conn)

            i_table = pandas.merge(left = fi_otc , right = tmp , on = 'no' , how = 'left')
            i_table = i_table[i_table['date_y'].isnull()].iloc[:,:3]
            
            for _ , data in i_table.iterrows():
                cur.execute(foreign_investor_sql,data)
            cur.execute('commit')
            
            i_table = ''
            
            print('【 Foreign Investor (otc) Over Sold 】{} data inserted.'.format(process_day))
            time.sleep(10)
        else:
            print('【 Foreign Investor (otc) Over Sold 】{} No data .'.format(process_day))


## 【 PythonOperator 】

In [21]:
task_GET_OTC_FI_OBS = PythonOperator(task_id = 'task_GET_OTC_FI_OBS' ,
                                     python_callable = OTC_FI_OBS ,
                                     dag = dag)

# GET_STOCK_IT_OBS

## 【 Function 】

In [22]:
def STOCK_IT_OBS():

    investment_trust_stock_url ='https://www.twse.com.tw/fund/TWT44U?response=html&date={:0>4}{:0>2}{:0>2}'
    investment_trust_sql = 'insert into investment_trust (date , no , quantity) values (%s,%s,%s)'
    
    work_D = [pandas.read_sql("select date from work_date order by date desc ",con = conn)['date'][0]]    
    
    for D in work_D:
        
        process_day = '{:0>4}-{:0>2}-{:0>2}'.format(D.year , D.month , D.day)
        
    # Investment Trust (Stock) Over Bought/Sold    
        
        try:
            process_day = '{:0>4}-{:0>2}-{:0>2}'.format(D.year,D.month,D.day)
            investment_trust = pandas.read_html(investment_trust_stock_url.format(D.year,D.month,D.day))[0].iloc[:,[1,5]]
            investment_trust.columns = ['no','quantity']
            investment_trust['no'] = investment_trust['no'].astype('string').str.pad(width = 4 , side = 'left' , fillchar = '0')
            investment_trust['store'] = investment_trust.apply(lambda x : 'y' if len(x['no']) == 4 else 'n' , axis = 1)
            investment_trust= investment_trust[investment_trust['store']=='y']
            investment_trust['date'] = process_day
            investment_trust = investment_trust.iloc[:,[3,0,1]]
            
            tmp = pandas.read_sql("select * from investment_trust where date = '{}'".format(process_day) , con = conn)
            
            i_table = pandas.merge(left = investment_trust , right = tmp , how = 'left' , on = 'no')
            i_table = i_table[i_table['date_y'].isnull()].iloc[:,:3]

            
            for _ , data in i_table.iterrows():
                cur.execute(investment_trust_sql,data)
            cur.execute('commit')
            
            i_table = ''
            
            print('【 Investment Trust (Stock) Over Bought / Sold 】{} data inserted .'.format(process_day))
            time.sleep(20)
            
        except ValueError:
            print('【 Investment Trust (Stock) Over Bought / Sold 】{} No data'.format(process_day))
            pass
        

## 【PythonOperator】

In [23]:
task_GET_STOCK_IT_OBS = PythonOperator(task_id = 'task_GET_STOCK_IT_OBS' ,
                                       python_callable = STOCK_IT_OBS ,
                                       dag = dag)

# GET_OTC_IT_OBS

## 【 Function 】

In [24]:
def OTC_IT_OBS():
    
    investment_trust_sql = 'insert into investment_trust (date , no , quantity) values (%s,%s,%s)'
    investment_trust_otc_sell_url = 'https://www.tpex.org.tw/web/stock/3insti/sitc_trading/sitctr_result.php?l=zh-tw&t=D&type=sell&d={:0>3}/{:0>2}/{:0>2}&o=htm'
    investment_trust_otc_buy_url = 'https://www.tpex.org.tw/web/stock/3insti/sitc_trading/sitctr_result.php?l=zh-tw&t=D&type=buy&d={:0>3}/{:0>2}/{:0>2}&o=htm'

    work_D = [pandas.read_sql("select date from work_date order by date desc ",con = conn)['date'][0]]    
    
    for D in work_D:
    
        process_day = '{:0>4}-{:0>2}-{:0>2}'.format(D.year , D.month , D.day)    
        
# Investment Trust (OTC) Over Bought

        it = pandas.read_html(investment_trust_otc_buy_url.format(D.year-1911,D.month,D.day))[0]
        if it.iloc[-1,0] != '共0筆':
            it = it.iloc[:-1,[1,5]]
            it.columns = ['no','quantity']
            it['no'] = it['no'].astype('string').str.pad(width = 4 , side = 'left' , fillchar = '0')
            it['quantity'] = it['quantity'].astype(int)*1000
            it['store'] = ''
            it['store'] = it.apply(lambda x : 'y' if len(x['no']) == 4 else 'n' , axis = 1)
            it = it[it['store'] == 'y']
            it['date'] = process_day
            it = it.iloc[:,[3,0,1]]
            
            tmp = pandas.read_sql("select * from investment_trust where date = '{}'".format(process_day) , con = conn)
            
            i_table = pandas.merge(left = it , right = tmp , how = 'left' , on = 'no')
            i_table = i_table[i_table['date_y'].isnull()].iloc[:,:3]

            for _ , data in i_table.iterrows():
                cur.execute(investment_trust_sql,data)
            cur.execute('commit')
            
            i_table = ''

            print('【 Investment Trust (Otc) Over Bought 】{} data inserted.'.format(process_day))
        else :
            print('【 Investment Trust (Otc) Over Bought 】{} No data .'.format(process_day))

# Investment True (OTC) Over Sold

        it = pandas.read_html(investment_trust_otc_sell_url.format(D.year-1911,D.month,D.day))[0]
        if it.iloc[-1,0] != '共0筆':
            it = it.iloc[:-1,[1,5]]
            it.columns = ['no','quantity']
            it['no'] = it['no'].astype('string').str.pad(width = 4 , side = 'left' , fillchar = '0')
            it['quantity'] = it['quantity'].astype(int)*1000
            it['store'] = ''
            it['store'] = it.apply(lambda x : 'y' if len(x['no']) == 4 else 'n' , axis = 1)
            it = it[it['store'] == 'y']
            it['date'] = process_day
            it = it.iloc[:,[3,0,1]]
            
            tmp = pandas.read_sql("select * from investment_trust where date = '{}'".format(process_day) , con = conn)
            
            i_table = pandas.merge(left = it , right = tmp , how = 'left' , on = 'no')
            i_table = i_table[i_table['date_y'].isnull()].iloc[:,:3]
            
            for _ , data in i_table.iterrows():
                cur.execute(investment_trust_sql,data)
            cur.execute('commit')
            
            i_table = ''
            
            print('【 Investment Trust (Otc) Over Sold 】{} data inserted.'.format(process_day))
            time.sleep(10)
        else :
            print('【 Investment Trust (Otc) Over Sold 】{} No data .'.format(process_day))

## 【 PythonOperator 】

In [25]:
task_GET_OTC_IT_OBS = PythonOperator(task_id = 'task_GET_OTC_IT_OBS' , 
                                     python_callable = OTC_IT_OBS ,
                                     dag = dag)

# GET_STOCK_DL_OBS

## 【 Function 】

In [26]:
def STOCK_DL_OBS():

    dealer_stock_url ='https://www.twse.com.tw/fund/TWT43U?response=html&date={:0>4}{:0>2}{:0>2}'
    dealer_sql = 'insert into dealer (date , no , quantity) values (%s,%s,%s)'
    
    work_D = [pandas.read_sql("select date from work_date order by date desc ",con = conn)['date'][0]]
    
    for D in work_D:    
    
        process_day = '{:0>4}-{:0>2}-{:0>2}'.format(D.year , D.month , D.day)        
    
        try:
            dealer = pandas.read_html(dealer_stock_url.format(D.year,D.month,D.day))[0]
            dealer = dealer.iloc[:,[0,10]]
            dealer.columns = ['no','quantity']
            dealer['no'] = dealer['no'].astype('string').str.pad(width = 4 , side = 'left' , fillchar = '0')
            dealer['store'] = dealer.apply(lambda x : 'y' if len(x['no']) == 4 else 'n' , axis=1)
            dealer = dealer[dealer['store']=='y']
            dealer = dealer[dealer['quantity'] != 0]
            dealer['date'] = process_day
            dealer = dealer.iloc[:,[3,0,1]]
            
            tmp = pandas.read_sql("select * from dealer where date = '{}'".format(process_day) , con = conn)
            
            i_table = pandas.merge(left = dealer , right = tmp , how = 'left' , on = 'no')
            i_table = i_table[i_table['date_y'].isnull()].iloc[:,:3]
            
            for _ , data in i_table.iterrows():
                cur.execute(dealer_sql,data)
            cur.execute('commit')
            
            i_table = ''

            print('【 Dealer (Stock) Over Bought / Sold 】{} data inserted .'.format(process_day))
            time.sleep(20)
            
        except ValueError:
            print('【 Dealer (Stock) Over Bought / Sold 】{} No data'.format(process_day))   
            pass

## 【 PythonOperator 】

In [27]:
task_GET_STOCK_DL_OBS = PythonOperator(task_id = 'task_GET_STOCK_DL_OBS' ,
                                       python_callable = STOCK_DL_OBS ,
                                       dag = dag)

# GET_OTC_DL_OBS

## 【 Function 】

In [28]:
def OTC_DL_OBS():
    
    dealer_otc_buy_url = 'https://www.tpex.org.tw/web/stock/3insti/dealer_trading/dealtr_hedge_result.php?l=zh-tw&o=htm&t=D&type=buy&d={:0>3}/{:0>2}/{:0>2}&s=0,asc'
    dealer_otc_sell_url = 'https://www.tpex.org.tw/web/stock/3insti/dealer_trading/dealtr_hedge_result.php?l=zh-tw&o=htm&t=D&type=sell&d={:0>3}/{:0>2}/{:0>2}&s=0,asc'
    dealer_sql = 'insert into dealer (date , no , quantity) values (%s,%s,%s)'

    work_D = [pandas.read_sql("select date from work_date order by date desc ",con = conn)['date'][0]]    
    
    for D in work_D:
        
        process_day = '{:0>4}-{:0>2}-{:0>2}'.format(D.year , D.month , D.day)    
    
# Dealer (OTC) Over Bought

        dealer = pandas.read_html(dealer_otc_buy_url.format(D.year-1911,D.month,D.day))[0]
        if dealer.iloc[-1,0] != '共0筆':
            dealer = dealer.iloc[:,[1,9]]
            dealer.columns = ['no','quantity']
            dealer['no'] = dealer['no'].astype('string').str.pad(width = 4 , side = 'left' , fillchar = '0')
            dealer['store'] = ''
            dealer['store'] = dealer.apply(lambda x : 'y' if len(x['no']) == 4 else 'n' , axis = 1)
            dealer = dealer[dealer['store'] == 'y']
            dealer['quantity'] = dealer['quantity'].astype(int)*1000
            dealer['date'] = process_day
            dealer = dealer.iloc[:,[3,0,1]]
            
            tmp = pandas.read_sql("select * from dealer where date = '{}'".format(process_day) , con = conn)
            
            i_table = pandas.merge(left = dealer , right = tmp , how = 'left' , on = 'no')
            i_table = i_table[i_table['date_y'].isnull()].iloc[:,:3]
            
            for _ , data in i_table.iterrows():
                cur.execute(dealer_sql,data)
            cur.execute('commit')
            
            i_table = ''
            
            print('【 Dealer (Otc) Over Bought 】{} data inserted.'.format(process_day))
        else:
            print('【 Dealer (Otc) Over Bought 】{} No data inserted.'.format(process_day))

    
# Dealer (OTC) Over Sold    
    
        dealer = pandas.read_html(dealer_otc_sell_url.format(D.year-1911,D.month,D.day))[0]
        if dealer.iloc[-1,0] != '共0筆':
            dealer = dealer.iloc[:,[1,9]]
            dealer.columns = ['no','quantity']
            dealer['no'] = dealer['no'].astype('string').str.pad(width = 4 , side = 'left' , fillchar = '0')
            dealer['store'] = ''
            dealer['store'] = dealer.apply(lambda x : 'y' if len(x['no']) == 4 else 'n' , axis = 1)
            dealer = dealer[dealer['store'] == 'y']
            dealer['quantity'] = dealer['quantity'].astype(int)*1000
            dealer['date'] = process_day
            dealer = dealer.iloc[:,[3,0,1]]
            
            tmp = pandas.read_sql("select * from dealer where date = '{}'".format(process_day) , con = conn)
            
            i_table = pandas.merge(left = dealer , right = tmp , how = 'left' , on = 'no')
            i_table = i_table[i_table['date_y'].isnull()].iloc[:,:3]
            
            for _ , data in i_table.iterrows():
                cur.execute(dealer_sql,data)
            cur.execute('commit')
            
            i_table = ''

            print('【 Dealer (Otc) Over Sold 】{} data inserted.'.format(process_day))
            time.sleep(10)
        else:
            print('【 Dealer (Otc) Over Sold 】{} No data .'.format(process_day))

## 【 PythonOperator 】

In [29]:
task_GET_OTC_DL_OBS = PythonOperator(task_id = 'task_GET_OTC_DL_OBS',
                                     python_callable = OTC_DL_OBS , 
                                     dag = dag)

# GET_OTC_ISSUED_AMOUNT

## 【 Function 】

In [30]:
def OTC_ISSUED_AMOUNTS():
    
    work_D = [pandas.read_sql("select date from work_date order by date desc ",con = conn)['date'][0]]
    
    for D in work_D:
        
        insert_D = '{:0>4}{:0>2}{:0>2}'.format(D.year,D.month,D.day)
        
        html = 'https://www.tpex.org.tw/web/stock/aftertrading/daily_close_quotes/stk_quote_result.php?l=zh-tw&o=htm&d={:0>3}/{:0>2}/{:0>2}&s=0,asc,0'.format(D.year-1911,D.month,D.day)
        try:
            main = pandas.read_html(html)[0].iloc[:,[0,15]]
            main['not_stock']= main.apply(lambda x : 'y' if x.iloc[0].isdigit() == True else 'n' , axis =1)
            main = main[main['not_stock'] == 'y'].iloc[:,[0,1]]
            main['not_etl']= main.apply(lambda x : 'y' if len(x.iloc[0]) == 4  else 'n' , axis =1 )
            main = main[main['not_etl'] == 'y'].iloc[:,[0,1]]
            main.columns = ['no','issued_amounts']
            #main = main[main['no'] != '8462']
            main['date'] = insert_D
            main['type'] = 'OTC'
            
            tmp = pandas.read_sql("select * from issued_amounts where type = 'OTC' and date = '{}'".format(insert_D) , con = conn)
            
            i_table = pandas.merge(left = main , right = tmp , how = 'left' , on = 'no')
            i_table = i_table[i_table['date_y'].isnull()].iloc[:,:4]

            
            for _ , data in i_table.iterrows():
                cur.execute("insert into issued_amounts (no , amounts , date , type) values (%s,%s,%s,%s)",data)
            cur.execute("commit")
            
            print('【 OTC issued amounts 】{} data inserted . '.format(insert_D))
            time.sleep(10)
        
        except ValueError:
            print('【 OTC issued amounts 】{} No data'.format(insert_D))
            pass

## 【 PythonOperator 】

In [31]:
task_GET_OTC_ISSUED_AMOUNT = PythonOperator(task_id = 'task_GET_OTC_ISSUED_AMOUNT' ,
                                            python_callable = OTC_ISSUED_AMOUNTS ,
                                            dag = dag)

# Full DAG

In [32]:
task_WORK_DATE >> task_CHECK_WORK_DATE >> task_NOT_TRADED_DATE 
task_CHECK_WORK_DATE >> task_GET_STOCK_DAILY >> task_GET_OTC_DAILY
task_GET_OTC_DAILY >> task_GET_STOCK_DAILY_AVG >> task_GET_OTC_DAILY_AVG 
task_GET_OTC_DAILY_AVG >> task_GET_STOCK_FI_OBS >> task_GET_OTC_FI_OBS
task_GET_OTC_FI_OBS >> task_GET_STOCK_IT_OBS >> task_GET_OTC_IT_OBS
task_GET_OTC_IT_OBS >> task_GET_STOCK_DL_OBS >> task_GET_OTC_DL_OBS >> task_GET_OTC_ISSUED_AMOUNT

<Task(PythonOperator): task_GET_OTC_ISSUED_AMOUNT>